In [ ]:
import csv    #for the cvs and txv files
import sqlite3 as sql   #for the db file
import pandas as pd    #Help visualize the data
import matplotlib.pyplot as plt
import resources.data_cleaning as dp
import resources.visualization as dv
%matplotlib inline

In [ ]:
df_1 = pd.read_csv("data/tmdb.movies.csv")   
df_2 = pd.read_csv("data/tn.movie_budgets.csv")
df_3 = pd.read_csv('data/rt.reviews.tsv',delimiter = '\t', encoding = 'cp437' )   
df_4 = pd.read_csv('data/rt.movie_info.tsv',delimiter = '\t')
df_5 = pd.read_csv('data/bom.movie_gross.csv',delimiter = '\t')
conn = sql.connect('data/im.db/im.db')

In [ ]:
df_1.info()    #Extra column to drop
df_1.duplicated().sum()    #No duplicate rows to drop
df_1.isna().sum()    #No NaN values to drop
df_1.isnull().sum()#

In [ ]:
df_2.info()    #Extra column to drop
print(df_2.duplicated().sum())    #No duplicate rows to drop
df_2.isna().sum()    #No NaN values to drop
df_2.isnull().sum()

In [ ]:
df_2['cat_f'] = df_2['prod_world_ratio'].map(lambda x: dp.cat_ratio(x))
zero_cat = len(df_2.loc[df_2['cat_f'] == 0])
one_cat = len(df_2.loc[df_2['cat_f'] == 1])
two_cat = len(df_2.loc[df_2['cat_f'] == 2])
three_cat = len(df_2.loc[df_2['cat_f'] == 3])

print("""
{:.0%} have a ratio of less than 1
{:.0%} have a ratio of 1 to 2
{:.0%} have a ratio of 2 to 3
{:.0%} have a ratio of more than 3
""".format(zero_cat/5782, one_cat/5782, two_cat/5782, three_cat/5782))

### Too screwed due to outliers?
print('Average production budget with ratio less than 1: ', df_2['production_budget'].loc[df_2['cat_f'] == 0].mean())
print('Average production budget with ratio between 1 and 2: ', df_2['production_budget'].loc[df_2['cat_f'] == 1].mean())
print('Average production budget with ratio between 2 and 3: ', df_2['production_budget'].loc[df_2['cat_f'] == 2].mean())
print('Average production budget with ratio over 3: ', df_2['production_budget'].loc[df_2['cat_f'] == 3].mean())

print('Average production budget with ratio less than 1: ', outlier_removed['production_budget'].loc[outlier_removed['cat_f'] == 0].mean())
print('Average production budget with ratio between 1 and 2: ', outlier_removed['production_budget'].loc[outlier_removed['cat_f'] == 1].mean())
print('Average production budget with ratio between 2 and 3: ', outlier_removed['production_budget'].loc[outlier_removed['cat_f'] == 2].mean())
print('Average production budget with ratio over 3: ', outlier_removed['production_budget'].loc[outlier_removed['cat_f'] == 3].mean())


In [ ]:
df_3.info()    #Extra column to drop
print(df_3.duplicated().sum())    #9 duplicate to drop
df_3.isna().sum()    #Review, rating, critics and publisher have NaN values
df_3.isnull().sum()    #Review, rating, critics and publisher have Null values

In [ ]:
df_4.info()    #Extra column to drop
print(df_4.duplicated().sum())    #No duplicate rows to drop
df_4.isna().sum()    #All columns but ID have NaN values
df_4.isnull().sum()    #All columns but ID have Null values

In [ ]:
df_5.info()    #Extra column to drop
print(df_5.duplicated().sum())    #No duplicate rows to drop
df_5.isna().sum()    #All columns but ID have NaN values
df_5.isnull().sum()    #There are no Null values
df_5.head()    #Looks the same as df_1 or rather similar data

In [ ]:
print(df_1.columns)
df_1['genre_ids']    #What does these numbers mean? - 
df_1['id']    #Each movie has a unique id
df_1['original_language']    #Only english movies? - No, there are multiples
df_1['original_title']     #Title of the movie - Use title for consistency
df_1['popularity']    #Ranking out of ... 100 , Website looks like percentage - based on amount timed watched?
df_1.sort_values('popularity').tail()
#Release data, title are pretty obvious, may need to look in title vs original title
df_1.loc[df_1['vote_average'] == 0]  #Quite a few with not votes and therefore no vote average

#Main use, productuon_budget, domestic gross, worldwide gross
#Main use, pull best and worst genre based on rating
#https://docs.python.org/3/library/codecs.html#standard-encodings - Helped with finding valid encoding
#Main use, studio, could use to double check df1
#SQL connection with movie ratings and director name

In [ ]:
# Rotten Tomatoes review info
df_3.head()
df_3.info() 
df_3.isna().sum()    #1/4 of ratings are NaN - may not be too useful
df_3.loc[df_3['review'].isna()]    #Review are just the written words - Okay to be NaN for our case
df3_top_critic = df_3.loc[df_3['top_critic'] == 1]    #1/4 of reviews are made from 'top critics' - Could be useful for creditabilty
#print(len(df3_top_critic['critic'].unique()))    #856 unique critics    
#print(len(df3_top_critic))    #13096 review from top critics

In [ ]:
### Find Average ratings for all the movies - 6.22
pd.read_sql("""SELECT AVG(averagerating) AS 'Average Rating'
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)
""", conn)

In [ ]:
pd.read_sql("""
SELECT AVG(NumberofMovies)
FROM (
    SELECT COUNT(movie_id) AS NumberofMovies
    FROM movie_ratings
    JOIN movie_basics
        USING (movie_id)
    JOIN directors
        USING (movie_id)
    JOIN persons
        USING (person_id)
    GROUP BY persons.primary_name
    HAVING NumberofMovies > 1
    )
    
""", conn)

In [ ]:
list_of_directors = dv.top_directors(conn)
best_dir_movies = []
for movie in list(list_of_directors['primary_title']):
    if movie in df1_top_list:
        best_dir_movies.append(movie)
        
best_dir_movies